In [1]:
# Step 1: Import necessary libraries
import pandas as pd
import sqlite3

In [2]:
# Set up dataframes

UNRATE = pd.read_csv('UNRATE.csv')
CPI = pd.read_csv('CPIAUCSL_CH1.csv')
FEDFUNDS = pd.read_csv('FEDFUNDS.csv')
GDP = pd.read_csv('GDPC1_CH1.csv')
CUNRATE = pd.read_csv('CGBD2534.csv')



In [3]:
# Create Database
conn = sqlite3.connect('database.db')
# Load DataFrames into Database as tables
UNRATE.to_sql('UNRATE', conn, if_exists='replace', index=False)
CPI.to_sql('CPI', conn, if_exists='replace', index=False)
FEDFUNDS.to_sql('FEDFUNDS', conn, if_exists='replace', index=False)
GDP.to_sql('GDP', conn, if_exists='replace', index=False)
CUNRATE.to_sql('CUNRATE', conn, if_exists='replace', index=False)

311

## 1. Basic SELECT Statement

The SELECT statement is used to query data from a database. The simplest form is SELECT * which retrieves all columns from a table.

In [4]:
# Select all columns and rows from the UNRATE table
query = """
SELECT * FROM UNRATE
"""

result = pd.read_sql_query(query, conn)
result.head()  # Show first 5 rows

,observation_date,UNRATE
0,1948-01-01,3.4
1,1948-02-01,3.8
2,1948-03-01,4.0
3,1948-04-01,3.9
4,1948-05-01,3.5


## 2. SELECT Specific Columns

Instead of selecting all columns with *, you can specify exactly which columns you want to retrieve.

In [5]:
# Select only specific columns
query = """
SELECT observation_date, UNRATE FROM UNRATE
"""

result = pd.read_sql_query(query, conn)
result.head()

,observation_date,UNRATE
0,1948-01-01,3.4
1,1948-02-01,3.8
2,1948-03-01,4.0
3,1948-04-01,3.9
4,1948-05-01,3.5


## 3. WHERE Clause - Filtering Data

The WHERE clause filters records based on specific conditions. Only rows that meet the condition are returned.

In [ ]:
# Select rows where unemployment rate is greater than 8%
#Note, if you want to filter out based on whether a row has a null value you can use "IS NULL" or "IS NOT NULL"
query = """
SELECT observation_date, UNRATE 
FROM UNRATE 
WHERE UNRATE > 8.0
"""

result = pd.read_sql_query(query, conn)
result

,observation_date,UNRATE
0,1975-01-01,8.1
1,1975-02-01,8.1
2,1975-03-01,8.6
3,1975-04-01,8.8
4,1975-05-01,9.0
...,...,...
81,2020-04-01,14.8
82,2020-05-01,13.2
83,2020-06-01,11.0
84,2020-07-01,10.2


In [17]:
#When using the Where function you can use logical operators, such as AND, OR, and NOT to combine multiple conditions.
# For example: If I wanted to find all months where the unemployment rate was greater than 8% or the federal funds rate was less than 2%, I could use the following query:
query = """
SELECT U.observation_date, U.UNRATE, F.FEDFUNDS
FROM UNRATE U
JOIN FEDFUNDS F ON U.observation_date = F.observation_date
WHERE U.UNRATE > 8.0 OR F.FEDFUNDS < 2.0
"""
result = pd.read_sql_query(query, conn)
result

,observation_date,UNRATE,FEDFUNDS
0,1954-07-01,5.8,0.80
1,1954-08-01,6.0,1.22
2,1954-09-01,6.1,1.07
3,1954-10-01,5.7,0.85
4,1954-11-01,5.3,0.83
...,...,...,...
255,2022-03-01,3.7,0.20
256,2022-04-01,3.7,0.33
257,2022-05-01,3.6,0.77
258,2022-06-01,3.6,1.21


## 4. ORDER BY - Sorting Results

ORDER BY sorts the result set in ascending (ASC) or descending (DESC) order based on one or more columns.

In [7]:
# Get unemployment rates sorted from highest to lowest
query = """
SELECT observation_date, UNRATE 
FROM UNRATE 
ORDER BY UNRATE DESC
"""

result = pd.read_sql_query(query, conn)
result.head(10)  # Show top 10 highest unemployment rates

,observation_date,UNRATE
0,2020-04-01,14.8
1,2020-05-01,13.2
2,2020-06-01,11.0
3,1982-11-01,10.8
4,1982-12-01,10.8
5,1982-10-01,10.4
6,1983-01-01,10.4
7,1983-02-01,10.4
8,1983-03-01,10.3
9,1983-04-01,10.2


## 5. LIMIT - Restricting Number of Results

LIMIT restricts the number of rows returned by the query. Useful for viewing a sample of data.

In [8]:
# Get only the first 5 records
query = """
SELECT observation_date, UNRATE 
FROM UNRATE 
LIMIT 5
"""

result = pd.read_sql_query(query, conn)
result

,observation_date,UNRATE
0,1948-01-01,3.4
1,1948-02-01,3.8
2,1948-03-01,4.0
3,1948-04-01,3.9
4,1948-05-01,3.5


## 6. BETWEEN - Range Filtering

BETWEEN is used to filter values within a specific range (inclusive).

In [9]:
# Find unemployment rates between 5% and 7%
query = """
SELECT observation_date, UNRATE 
FROM UNRATE 
WHERE UNRATE BETWEEN 5.0 AND 7.0
"""

result = pd.read_sql_query(query, conn)
result.head()

,observation_date,UNRATE
0,1949-03-01,5.0
1,1949-04-01,5.3
2,1949-05-01,6.1
3,1949-06-01,6.2
4,1949-07-01,6.7


## 7. INNER JOIN - Combining Tables

INNER JOIN returns only the rows where there is a match in both tables based on the join condition.

In [10]:
# This query selects the dates where the federal funds rate is greater than 5.0 along with the corresponding unemployment rates.

query = """
SELECT UNRATE.observation_date, UNRATE.UNRATE, FEDFUNDS.FEDFUNDS
FROM UNRATE
INNER JOIN FEDFUNDS ON UNRATE.observation_date = FEDFUNDS.observation_date 
WHERE FEDFUNDS.FEDFUNDS > 5.0
"""

result = pd.read_sql_query(query, conn)
result

,observation_date,UNRATE,FEDFUNDS
0,1966-06-01,3.8,5.17
1,1966-07-01,3.8,5.30
2,1966-08-01,3.8,5.53
3,1966-09-01,3.7,5.40
4,1966-10-01,3.7,5.53
...,...,...,...
357,2024-05-01,4.0,5.33
358,2024-06-01,4.1,5.33
359,2024-07-01,4.2,5.33
360,2024-08-01,4.2,5.33


## 8. LEFT JOIN - Keeping All Records from Left Table

LEFT JOIN returns all rows from the left table and matching rows from the right table. If no match exists, NULL values are returned for the right table columns.

In [11]:
# This is a query that left joins the UNRATE and CUNRATE tables on the observation_date column, retrieving all records from the UNRATE table along with matching records from the CUNRATE table.

query = """
SELECT UNRATE.observation_date, UNRATE.UNRATE, CUNRATE.CGBD2534 
FROM UNRATE 
LEFT JOIN CUNRATE ON UNRATE.observation_date = CUNRATE.observation_date
"""

result = pd.read_sql_query(query, conn)
result

,observation_date,UNRATE,CGBD2534
0,1948-01-01,3.4,NaN
1,1948-02-01,3.8,NaN
2,1948-03-01,4.0,NaN
3,1948-04-01,3.9,NaN
4,1948-05-01,3.5,NaN
...,...,...,...
930,2025-07-01,4.2,3.8
931,2025-08-01,4.3,3.6
932,2025-09-01,4.4,3.6
933,2025-10-01,NaN,NaN


## 9. Aggregate Functions - COUNT, AVG, MIN, MAX, SUM

Aggregate functions perform calculations on a set of values and return a single value.

In [12]:
# Calculate statistics on unemployment rate
query = """
SELECT 
    COUNT(*) as total_records,
    AVG(UNRATE) as average_unemployment,
    MIN(UNRATE) as min_unemployment,
    MAX(UNRATE) as max_unemployment
FROM UNRATE
"""

result = pd.read_sql_query(query, conn)
result

,total_records,average_unemployment,min_unemployment,max_unemployment
0,935,5.66788,2.5,14.8


## 10. GROUP BY - Aggregating Data by Categories

GROUP BY groups rows that have the same values in specified columns, often used with aggregate functions.

In [ ]:
# Calculate average federal funds rate by year
#Note the use of substring to extract the year from the observation_date
query = """
SELECT 
    SUBSTR(observation_date, 1, 4) as year,
    AVG(FEDFUNDS) as avg_fed_funds_rate,
    COUNT(*) as num_observations
FROM FEDFUNDS
GROUP BY year
ORDER BY year DESC
"""
#also note how DESC is used to order the years from most recent to oldest

result = pd.read_sql_query(query, conn)
result.head(10)

,year,avg_fed_funds_rate,num_observations
0,2025,4.212500,12
1,2024,5.143333,12
2,2023,5.024167,12
3,2022,1.683333,12
4,2021,0.080000,12
5,2020,0.375833,12
6,2019,2.158333,12
7,2018,1.831667,12
8,2017,1.001667,12
9,2016,0.395000,12


# NTILE function

The NTILE function allows you to group your data and add a categorical piece to a normally numeric Data set

In [14]:
#For example, let us take the unemployment rate and we want to categorize it into quartiles, this is how we 
#Would use the NTILE function in SQL to achieve that.
querey = """
SELECT 
    UNRATE.observation_date as date, 
    UNRATE.UNRATE as rate,
    NTILE(4) OVER (ORDER BY UNRATE.UNRATE) as quartile
FROM UNRATE
ORDER BY UNRATE.UNRATE
"""
result = pd.read_sql_query(querey, conn)
result

,date,rate,quartile
0,2025-10-01,NaN,1
1,1953-05-01,2.5,1
2,1953-06-01,2.5,1
3,1953-02-01,2.6,1
4,1953-03-01,2.6,1
...,...,...,...
930,1982-11-01,10.8,4
931,1982-12-01,10.8,4
932,2020-06-01,11.0,4
933,2020-05-01,13.2,4


In [15]:
#now lets say you want to get rid of Null variables and want to reorganize it by date, this is what you would do:
query = """
SELECT 
    UNRATE.observation_date as date, 
    UNRATE.UNRATE as rate,
    NTILE(4) OVER (ORDER BY UNRATE.UNRATE) as quartile
FROM UNRATE
WHERE UNRATE.UNRATE IS NOT NULL
ORDER BY UNRATE.observation_date
"""
result = pd.read_sql_query(query, conn)
result

,date,rate,quartile
0,1948-01-01,3.4,1
1,1948-02-01,3.8,1
2,1948-03-01,4.0,1
3,1948-04-01,3.9,1
4,1948-05-01,3.5,1
...,...,...,...
929,2025-06-01,4.1,1
930,2025-07-01,4.2,1
931,2025-08-01,4.3,2
932,2025-09-01,4.4,2
